In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score


In [4]:
with open('public_api_results/cohere.tsv', 'r') as f:
    ch_df = pd.read_csv(f, sep='\t')
with open('public_api_results/openai.tsv', 'r') as f:
    ai_df = pd.read_csv(f, sep='\t')

In [7]:
partition_col = 'prompt_len'
independent_col = 'output_len'

def save_figs(partition_col, independent_col, df):
    model_names = set(df['model_name'])
    prompt_lens = set(ch_df[partition_col])
    fig, ax = plt.subplots(figsize = (9, 9))
    plt.title(f"Call latency v. output length for public inference APIs")
    plt.xlabel("Output length")
    plt.ylabel("Call latency (seconds)")
    print(model_names)
    for (model, color) in zip(model_names, ['r', 'b', 'g', 'c', 'k', 'm', 'y']):
        subset = df[df.model_name == model]

        x, y = subset[independent_col], subset['call_latency']
        ax.scatter(x,y, color=color, label=model, s=10)

        b, a = np.polyfit(x, y, deg=1)
        predict = np.poly1d((b,a))
        r2 = r2_score(y, predict(x))
        print(f"Model {model}, r2={r2}")
        # Create sequence of 100 numbers from 0 to 100 
        xseq = np.linspace(min(df[independent_col]),max(df[independent_col]), num=100)

        # Plot regression line
        ax.plot(xseq, a + b * xseq, color=color, lw=1);
    plt.legend(loc="upper left")

            
    plt.savefig(f"public_api_results/latencies.png")
    plt.clf()

save_figs(partition_col, independent_col, pd.concat([ai_df, ch_df]))


    

{'davinci-instruct-beta', 'command-medium-nightly', 'command-xlarge-nightly', 'text-curie-001', 'text-babbage-001', 'text-davinci-003', 'gpt-3.5-turbo'}
Model davinci-instruct-beta, r2=0.9993864720194614
Model command-medium-nightly, r2=0.38265638980811545
Model command-xlarge-nightly, r2=0.5755382521348713
Model text-curie-001, r2=0.963909850761274
Model text-babbage-001, r2=0.9424462994608613
Model text-davinci-003, r2=0.9940426839652995
Model gpt-3.5-turbo, r2=0.9758958822036207


<Figure size 900x900 with 0 Axes>